## Accessing DM-EFD data


In this notebook we demonstrate how to extract data from the DM-EFD using [aioinflux](https://aioinflux.readthedocs.io/en/stable/index.html), a Python client for InfluxDB, and proceed with data analysis using Pandas dataframes. 

This is complementaty to the [Chronograf](https://test-chronograf-efd.lsst.codes) interface which we use for time-series visualization.

In addition to `aioinflux`, you'll need to install `pandas`, `numpy` and `matplotlib` to run this notebook.

In [ ]:
import matplotlib
%matplotlib widget
from matplotlib import pylab as plt
import pandas as pd
import asyncio

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d
output_notebook()

We'll access the DM-EFD instance deployed at the AuxTel lab in Tucson. You need to be on site or connected to the NOAO VPN. 

If you are familiar with the AuxTel lab environment, you might be able to authenticate using the generic `saluser`. Ping me at Slack (`@afausti`) if you have any problem.

In [ ]:
import jupyterlabutils.notebook as jn

In [ ]:
efd_client = jn.EfdClient('summit_efd')

In [ ]:
topics = await efd_client.get_topics()

In [ ]:
fields = await efd_client.get_fields(topics[0])

In [ ]:
# Must be run on the Lab EFD
result = await efd_client.select_time_series('lsst.sal.ATCamera.wreb', '* ',
                                       pd.Timestamp('2019-09-08T01:41:00Z'), pd.Timedelta('3m'))

In [ ]:
start = pd.Timestamp('2019-11-26T15:48:00Z')
end = pd.Timedelta('9.5m')

In [ ]:
measured_azel = await efd_client.select_packed_time_series('lsst.sal.ATMCS.mount_AzEl_Encoders', ['azimuthCalculatedAngle', 'elevationCalculatedAngle'], start, end)
measured_azel_vel = await efd_client.select_packed_time_series('lsst.sal.ATMCS.measuredMotorVelocity', ['azimuthMotor1Velocity', 'azimuthMotor2Velocity', 'elevationMotorVelocity'], start, end)
commanded_azel = await efd_client.select_packed_time_series('lsst.sal.ATMCS.trajectory', ['azimuth', 'azimuthVelocity', 'elevation', 'elevationVelocity'], start, end)

In [ ]:
p = figure(x_axis_type='datetime', plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_azel.index.values, y=commanded_azel['azimuth'], color='black', line_width=2, legend_label='Commanded Az')
p.line(x=measured_azel.index.values, y=measured_azel['azimuthCalculatedAngle'], color='lightblue', line_width=2, legend_label='Computed Az')
p.extra_y_ranges = {'Velocity': Range1d(start=-2.5, end=2.5)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=measured_azel_vel.index.values, y=measured_azel_vel['azimuthMotor1Velocity'], color='red', alpha=0.5, y_range_name='Velocity', legend_label='Measured Az Velocity Motor 1')
p.line(x=measured_azel_vel.index.values, y=measured_azel_vel['azimuthMotor2Velocity'], color='blue', alpha=0.5, y_range_name='Velocity', legend_label='Measured Az Velocity Motor 2')
p.line(x=commanded_azel.index.values, y=commanded_azel['azimuthVelocity'], color='black', alpha=0.5, y_range_name='Velocity', legend_label='Commanded Az Velocity')
p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

In [ ]:
interp = jn.resample(measured_azel, commanded_azel)
ratio = interp['elevationCalculatedAngle']/interp['elevation']

In [ ]:
p = figure(x_axis_type='datetime', y_range=(0.99999, 1.00001), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Ratio of measured to commanded elevation"
p.xaxis.axis_label = "Time"
p.extra_y_ranges = {'Velocity': Range1d(start=-0.1, end=0.1)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=measured_azel_vel.index.values, y=measured_azel_vel['elevationMotorVelocity'], color='red', alpha=0.5, y_range_name='Velocity', legend_label='Measured El Velocity Motor')
p.line(x=commanded_azel.index.values, y=commanded_azel['elevationVelocity'], color='black', alpha=0.5, y_range_name='Velocity', legend_label='Commanded El Velocity')
p.line(x=ratio.index.values, y=ratio.values, color='black', line_width=2, legend_label='ratio', alpha=0.3)
p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)